Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Competition Imports

In [240]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC, NuSVC

# Latent Semantic Indexing
from sklearn.decomposition import TruncatedSVD

# Word Embeddings
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

from xgboost import XGBClassifier

import numpy as np

### Load Competition Data

In [29]:
import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [30]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['ratingCategory'], random_state=42)

train.shape, val.shape, test.shape

((3269, 3), (818, 3), (1022, 2))

In [31]:
val.head()

,id,description,ratingCategory
2368,62,\nAn essay in balance on both the aroma and pa...,0
3794,2789,\nAn Islay single malt of unknown origin aged ...,1
1380,584,\nA quintet of releases showing examples of Th...,1
239,4000,"\n100% rye, unmalted and malted, direct-fire p...",0
436,2520,\nThis was aged in a refill American oak hogsh...,1


### Dataframe cleanup

In [35]:
def wrangle(X):
    
    X = X.copy()
    
    return X

In [36]:
train = wrangle(train)
val = wrangle(val)

In [38]:
# Split train data
y_train = train['ratingCategory']
X_train = train['description']

y_val = val['ratingCategory']
X_val = val['description']

X_train.shape, X_val.shape

((3269,), (818,))

### Define Pipeline Components - Global

In [230]:
# Pipeline components
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
# clf = RandomForestClassifier()
clf = SGDClassifier(random_state=42)

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Search Space (GridSearch)
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [231]:
parameters_grid = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500, 1000),
    # 'clf__n_estimators':(5, 10),
    # 'clf__max_depth':(15, 20)
}

grid_search = GridSearchCV(pipe, parameters_grid, cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print ('Train Accuracy', grid_search.score(X_train, y_train))
print ('Validation Accuracy', grid_search.score(X_val, y_val))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.6s finished


Train Accuracy 0.795962067910676
Validation Accuracy 0.7420537897310513


In [232]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.72897050077038
{'vect__max_df': 0.75, 'vect__max_features': 500, 'vect__min_df': 0.02}


### Define search space (RandomCV)

In [179]:
parameters_random = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': range(500, 5000, 200),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}


random_search = RandomizedSearchCV(pipe, param_distributions=parameters_random, n_iter = 20, cv = 3, 
                                   verbose=1, random_state=42, n_jobs=-1)

# Fit the random search model
random_search.fit(X_train, y_train)

print ('Train Accuracy', random_search.score(X_train, y_train))
print ('Validation Accuracy', random_search.score(X_val, y_val))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    8.3s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [180]:
print(random_search.best_score_)
print(random_search.best_params_)

0.7207131082860858
{'vect__min_df': 0.02, 'vect__max_features': 4100, 'vect__max_df': 0.75, 'clf__n_estimators': 10, 'clf__max_depth': 20}


### Define Search Space (Assignment)

In [224]:
# Lecture pipeline components

vect_assignment_parameters = {
    'stop_words': 'english',
    'ngram_range': (1, 2),
    'max_df': 0.75,
    'min_df': 0.02,
    'max_features': 500,
}

nusvc_assignment_parameters = {
    'nu': 0.005,
    'kernel': 'linear',
    'random_state': 42
}

vect_assignment = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf_assignment = RandomForestClassifier()

pipe_assignment = Pipeline([('vect', vect_assignment), ('clf', clf_assignment)])

In [225]:
pipe_assignment.fit(X_train, y_train)

print ('Train Accuracy', pipe_assignment.score(X_train, y_train))
print ('Validation Accuracy', pipe_assignment.score(X_val, y_val))

Train Accuracy 1.0
Validation Accuracy 0.7347188264058679


### Define Pipeline Components (Grid/Random Results)

In [185]:
vect_parameters_post = {
    'max_df': 0.75,
    'min_df': 0.02,
    'max_features': 500,
}

clf_parameters_post = {
    'n_estimators': 100,
    'max_depth': 20
}

vect_post = TfidfVectorizer(stop_words='english', ngram_range=(1,2), **vect_parameters_post)
clf_post = RandomForestClassifier(**clf_parameters_post)

pipe_post = Pipeline([('vect', vect_post), ('clf', clf_post)])

### Fitting Model (Accuracy)

In [193]:
random_search.fit(X_train, y_train)

print ('Train Accuracy', random_search.score(X_train, y_train))
print ('Validation Accuracy', random_search.score(X_val, y_val))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    8.1s finished


Train Accuracy 0.8617314163352707
Validation Accuracy 0.7188264058679706


### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [233]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [234]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [235]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [236]:
subNumber = 0

In [237]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
---
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
vect_indexing = TfidfVectorizer(stop_words='english')
svd_indexing = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)
rfc_indexing = RandomForestClassifier()

# LSI
lsi_indexing = Pipeline([('vect', vect_indexing), ('svd', svd_indexing)])

# Pipe
pipeline_indexing = Pipeline([('lsi', lsi_indexing), ('clf', rfc_indexing)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75,1.0),
    'lsi__vect__ngram_range': [(1,1),(1,2),(1,3)],
    'clf__n_estimators': (5,10,15,20)
}

grid_search = GridSearchCV(pipeline_indexing, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   12.7s


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
___
<a id="p3"></a>

## Follow Along

In [1]:
param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?